In [1]:
from PIL import Image, ImageEnhance, ImageFilter, ImageTransform
import random
from collections import Counter
import numpy as np

In [17]:

air = ["air" + str(i) + '.png' for i in range(1,4)]
dark = ["dark" + str(i) + '.png' for i in range(1,4)]
earth = ["earth" + str(i) + '.png' for i in range(1,4)]
fire = ["fire" + str(i) + '.png' for i in range(1,4)]
life = ["life" + str(i) + '.png' for i in range(1,4)]
light = ["light" + str(i) + '.png' for i in range(1,4)]
poison = ["poison" + str(i) + '.png' for i in range(1,4)]
reflfection = ["reflection" + str(i) + '.png' for i in range(1,4)]
water = ["water" + str(i) + '.png' for i in range(1,4)]


In [3]:

def slightly_change_image(image_path, output_path):
    # Open the image
    img = Image.open(image_path)
    
    # Randomly select which modifications to apply
    modifications = random.sample([
        'brightness', 'contrast', 'color', 'sharpness', 'blur', 'rotate'
    ], random.randint(1, 3))
    
    # Apply selected modifications
    for mod in modifications:
        if mod == 'brightness':
            enhancer = ImageEnhance.Brightness(img)
            img = enhancer.enhance(random.uniform(0.9, 1.5))
        elif mod == 'contrast':
            enhancer = ImageEnhance.Contrast(img)
            img = enhancer.enhance(random.uniform(0.8, 1.1))
        elif mod == 'color':
            enhancer = ImageEnhance.Color(img)
            img = enhancer.enhance(random.uniform(0.8, 1.2))
        elif mod == 'sharpness':
            enhancer = ImageEnhance.Sharpness(img)
            img = enhancer.enhance(random.uniform(0.8, 1.3))
        elif mod == 'blur':
            img = img.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.1, 0.7)))
        elif mod == 'rotate':
            img = img.rotate(random.uniform(-10, 10), expand=True)
    
    # Save the modified image
    img.save(output_path)

In [4]:
from PIL import Image, ImageOps, ImageFilter, ImageChops
import numpy as np
import random

def radical_pixel_transform(image_path, output_path):
    img = Image.open(image_path)
    
    # Convert to numpy array for manipulation
    arr = np.array(img)
    
    # Extreme color shifts
    arr[:,:,0] = arr[:,:,0] * random.uniform(0.1, 2.0)  # Red channel
    arr[:,:,1] = arr[:,:,1] * random.uniform(0.1, 2.0)  # Green channel
    arr[:,:,2] = arr[:,:,2] * random.uniform(0.1, 2.0)  # Blue channel
    
    # Pixel sorting (sort rows by brightness)
    for i in range(arr.shape[0]):
        brightness = arr[i,:,0] * 0.299 + arr[i,:,1] * 0.587 + arr[i,:,2] * 0.114
        sorted_indices = np.argsort(brightness)
        arr[i,:,:] = arr[i,sorted_indices,:]
    
    # Convert back to image
    img = Image.fromarray(np.uint8(arr))
    
    # Apply extreme filters
    img = img.filter(ImageFilter.EDGE_ENHANCE_MORE)
    img = ImageOps.posterize(img, random.randint(1, 4))
    img = ImageOps.solarize(img, threshold=random.randint(64, 192))
    
    # Random geometric transformations
    if random.random() > 0.5:
        img = img.rotate(random.randint(15, 345), expand=True)
    if random.random() > 0.5:
        img = img.transform(img.size, Image.AFFINE, 
                          (1, random.uniform(-0.5, 0.5), 0, 
                           random.uniform(-0.5, 0.5), 1, 0))
    
    # Save the result
    img.save(output_path)


In [18]:
figures = air + fire + dark + water + light + poison + reflfection + life + earth
figures

['air1.png',
 'air2.png',
 'air3.png',
 'fire1.png',
 'fire2.png',
 'fire3.png',
 'dark1.png',
 'dark2.png',
 'dark3.png',
 'water1.png',
 'water2.png',
 'water3.png',
 'light1.png',
 'light2.png',
 'light3.png',
 'poison1.png',
 'poison2.png',
 'poison3.png',
 'reflection1.png',
 'reflection2.png',
 'reflection3.png',
 'life1.png',
 'life2.png',
 'life3.png',
 'earth1.png',
 'earth2.png',
 'earth3.png']

In [19]:
figures = air + fire + dark + water + light + poison + reflfection + life + earth

cnt = Counter(figures)
for i in range(1000):
    figure = figures[random.randint(0,26)]
    cnt[figure] += 1
    choose = random.randint(1,20)
    if choose== 10:
        radical_pixel_transform('etalon/' + figure, 'images/' + 'x' + str(cnt[figure]) + '.png')
    else:
        slightly_change_image('etalon/' + figure, 'images/'  + figure[:figure.find('.')] +'-' +str(cnt[figure]) + '.png')

In [38]:
import tensorflow as tf
import numpy as np
class_names = {0: 'circle', 1: 'rectangle', 2: 'unknown'}
img_path = 'circle.png'
img = Image.open(img_path).convert('L')  # Convert to grayscale
img = img.resize((64, 64))  # Resize to match training size
img_array = np.array(img) / 255.0  # Normalize pixel values
img_array = np.expand_dims(img_array, axis=-1)  # Add channel dimension
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

model = tf.keras.models.load_model('high_confidence_model.h5')

predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])
confidence = np.max(predictions[0])
predicted_class_name = class_names[predicted_class]
    
# Print results
print(f"\nPrediction results for {img_path}:")
print(f"Class: {predicted_class_name} (confidence: {confidence:.2%})")
print("\nClass probabilities:")
for i, prob in enumerate(predictions[0]):
    print(f"{class_names[i]}: {prob:.2%}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step

Prediction results for circle.png:
Class: rectangle (confidence: 100.00%)

Class probabilities:
circle: 0.00%
rectangle: 100.00%
unknown: 0.00%
